In [ ]:
import time
import numpy as np
import pandas as pd
from datetime import datetime

from datetime import datetime
import fxcmpy
from sklearn import svm
import time

In [ ]:
TOKEN =''
con = fxcmpy.fxcmpy(access_token=TOKEN, log_level='error')

con.is_connected()

In [ ]:
fxcmpy.__version__

In [ ]:
to_show = ['tradeId', 'amountK','currency','grossPL','isBuy']

In [ ]:
lags = 3

In [ ]:
model = svm.SVC(C=100, probability=True)

In [ ]:
def generate_features(df, lags):
    df['Returns'] = np.log(df['Mid'] /df['Mid'].shift(1))
    print(df)
    cols = []
    for lag in range(1, lags +1):
        col = 'lag_%s' % lag
        print("col = ", col)
        df[col] = np.sign(df['Returns'].shift(lag))
        cols.append(col)
    df.dropna(inplace=True)
    return df, cols

In [ ]:
candles = con.get_candles('EUR/USD', period='m1', number=1000)

In [ ]:
data = pd.DataFrame(candles[['askclose', 'bidclose']].mean(axis=1),
                   columns=['Mid'])

In [ ]:
data, cols = generate_features(data, lags)
data.tail()

In [ ]:
model.fit(data[cols], np.sign(data['Returns']))

In [ ]:
ticks=0
position=0
tick_data=pd.DataFrame()
tick_resam=pd.DataFrame()

In [ ]:
def automated_trading(data,df):
    global lags,model,ticks
    global tick_data,tick_resan,to_show
    global position
    ticks += 1
    t =datetime.now()
    if ticks % 5==0 :
        print('%3d|%s| %7.5f | %7.5f' % (ticks, str(t.time()),
                                         data['Rates'][0],data['Rates'][1]))
    
    #collecting tick data    
    tick_data=tick_data.append(pd.DataFrame(
        {'Bid':data['Rates'][0],'Ask':data['Rates'][1],
         'High':data['Rates'][2],'Low':data['Rates'][3]},
         index=[t]))  
    
    #resampling tick data
    tick_resam=tick_data[['Bid','Ask']].resample('5s',label='right').last().ffill()
    tick_resam['Mid'] = tick_resam.mean(axis=1)
    time.sleep(1)
    if len(tick_resam) > lags + 2:
        #generate a signal
        tick_resam, cols = generate_features(tick_resam, lags)
        tick_resam['Prediction'] = model.predict(tick_resam[cols])
        #entering a long position 
        if tick_resam['Prediction'].iloc[-2] >= 0 and position == 0:
            print('going long (for the first time)')
            position = 1
            order = con.create_market_buy_order('EUR/USD', 25)
            trade = True 
        elif tick_resam['Prediction'].iloc[-2] >= 0 and position == -1:
            print('going long')
            position = 1
            order = con.create_market_buy_order('EUR/USD', 50)
            trade = True
        #entering a short positon 
        elif tick_resam['Prediction'].iloc[-2] <= 0 and position == 0:
            print('going short (for the first time)')
            position = -1
            order = con.create_market_buy_order('EUR/USD', 25)
            trade = True
        elif tick_resam['Prediction'].iloc[-2] <= 0 and position == 1:
            print('going short')
            position = -1
            order = con.create_market_buy_order('EUR/USD', 50)
            trade = True

In [ ]:
con.subscribe_market_data('EUR/USD', (automated_trading,))